In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import requests
from bs4 import BeautifulSoup
import json
import time
import copy

In [2]:
url = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2019&type=reg&cat=T&group=D&conf='
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Host": "www.footballdb.com",
        "Referer": "https://www.footballdb.com/stats/teamstat.html?group=D&cat=T"
    }
r = requests.get(url,headers=headers)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>\n  <title>\n   2019 NFL Team Defense Statistics | The Football Database\n  </title>\n  <meta content="View the 2019 NFL Team Defense statistical leaders as well as team statistics and stats from past NFL seasons." name="description"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="FootballDB.com" property="og:site_name"/>\n  <meta content="2019 NFL Team Defense Statistics | The Football Database" property="og:title">\n   <meta content="website" property="og:type"/>\n   <meta content="View the 2019 NFL Team Defense statistical leaders as well as team statistics and stats from past NFL seasons." property="og:description"/>\n   <meta content="https://www.footballdb.com/images/footballdb_200x200.png" property="og:image"/>\n   <meta content="https://www.footballdb.com/sta

In [5]:
tables = soup.find_all('table')

In [6]:
tables

[<table class="statistics scrollable">
 <thead>
 <tr class="header right">
 <th class="left"><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=teamname" title="Team">Team</a></th>
 <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=games" title="Games">Gms</a></th>
 <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=points" title="Total Points Scored">Tot Pts</a></th>
 <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=ptsgm" title="Points Per Game">Pts/G</a></th>
 <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushyds" title="Total Team Rushing Yards">RushYds</a></th>
 <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushypg" title="Rushing Yards Per G

In [7]:
col_names = soup.find_all('th')
row_names = soup.find_all('tr')
col_names, row_names

([<th class="left"><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=teamname" title="Team">Team</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=games" title="Games">Gms</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=points" title="Total Points Scored">Tot Pts</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=ptsgm" title="Points Per Game">Pts/G</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushyds" title="Total Team Rushing Yards">RushYds</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2019&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushypg" title="Rushing Yards Per Game">RYds/G</a></th>,
  <th><a href="/stats/teamstat.html?lg=NFL

In [8]:
columns = {}
for col in col_names:
    columns[col.text] = None
print(columns)

{'Team': None, 'Gms': None, 'Tot\xa0Pts': None, 'Pts/G': None, 'RushYds': None, 'RYds/G': None, 'PassYds': None, 'PYds/G': None, 'TotYds': None, 'Yds/G': None}


In [12]:
dict_data = []
keys = list(columns.keys())
for i, row in enumerate(row_names):
    if i > 0:
        new_row = copy.copy(columns)
        entries = row.find_all('td')
        for j, entry in enumerate(entries):
            new_row[keys[j]] = entry.text
        dict_data.append(new_row)
dict_data

[{'Team': 'New England PatriotsNew England',
  'Gms': '16',
  'Tot\xa0Pts': '225',
  'Pts/G': '14.1',
  'RushYds': '1528',
  'RYds/G': '95.5',
  'PassYds': '2886',
  'PYds/G': '180.4',
  'TotYds': '4414',
  'Yds/G': '275.9'},
 {'Team': 'San Francisco 49ersSan Francisco',
  'Gms': '16',
  'Tot\xa0Pts': '310',
  'Pts/G': '19.4',
  'RushYds': '1802',
  'RYds/G': '112.6',
  'PassYds': '2707',
  'PYds/G': '169.2',
  'TotYds': '4509',
  'Yds/G': '281.8'},
 {'Team': 'Buffalo BillsBuffalo',
  'Gms': '16',
  'Tot\xa0Pts': '259',
  'Pts/G': '16.2',
  'RushYds': '1649',
  'RYds/G': '103.1',
  'PassYds': '3123',
  'PYds/G': '195.2',
  'TotYds': '4772',
  'Yds/G': '298.2'},
 {'Team': 'Baltimore RavensBaltimore',
  'Gms': '16',
  'Tot\xa0Pts': '282',
  'Pts/G': '17.6',
  'RushYds': '1494',
  'RYds/G': '93.4',
  'PassYds': '3315',
  'PYds/G': '207.2',
  'TotYds': '4809',
  'Yds/G': '300.6'},
 {'Team': 'Pittsburgh SteelersPittsburgh',
  'Gms': '16',
  'Tot\xa0Pts': '303',
  'Pts/G': '18.9',
  'RushYds

In [13]:
fb_df = pd.DataFrame(dict_data)
fb_df

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
0,New England PatriotsNew England,16,225,14.1,1528,95.5,2886,180.4,4414,275.9
1,San Francisco 49ersSan Francisco,16,310,19.4,1802,112.6,2707,169.2,4509,281.8
2,Buffalo BillsBuffalo,16,259,16.2,1649,103.1,3123,195.2,4772,298.2
3,Baltimore RavensBaltimore,16,282,17.6,1494,93.4,3315,207.2,4809,300.6
4,Pittsburgh SteelersPittsburgh,16,303,18.9,1753,109.6,3113,194.6,4866,304.1
5,Los Angeles ChargersLA Chargers,16,345,21.6,1805,112.8,3204,200.2,5009,313.1
6,New York JetsNY Jets,16,359,22.4,1391,86.9,3779,236.2,5170,323.1
7,Chicago BearsChicago,16,298,18.6,1632,102.0,3554,222.1,5186,324.1
8,Dallas CowboysDallas,16,321,20.1,1656,103.5,3576,223.5,5232,327.0
9,Philadelphia EaglesPhiladelphia,16,354,22.1,1442,90.1,3865,241.6,5307,331.7
